In [78]:
import os
from dotenv import load_dotenv
import anthropic
import gradio as gr


load_dotenv()

api_key = os.getenv('ANTHROPIC_API_KEY').strip()
if not api_key.startswith('sk-ant-'):
    raise ValueError("API key appears to be in incorrect format. Should start with 'sk-ant-'")

with open('system_message.txt', 'r') as file:
    system_message = file.read()

if not system_message:
    raise ValueError(
        "system_message file could not be read"
    )
    
client = anthropic.Anthropic(api_key=api_key)

In [79]:
def stream_claude_3_7_sonnet(prompt):
    result = client.messages.stream(
        model="claude-3-7-sonnet-20250219",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    
    response = ""
    
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [80]:
with gr.Blocks() as chat:
    
    first_load = gr.State(True)
    default_language = gr.State("English")  # Store the current language
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    # Language selector at the top
    with gr.Row():
        submit_btn = gr.Button("Submit")
        
    with gr.Row():
        language_dropdown = gr.Dropdown(
            choices=["English", "Swedish", "Norwegian", "French", "German", "Danish","Finnish"], 
            value="English",
            label="Select Language",
            interactive=True
        )
        set_language_btn = gr.Button("Set Language")
        translate_button = gr.Button("Translate Last Response")
        clear = gr.Button("Clear")
    
    def user(user_message, history):
        return "", history + [[user_message, None]]
    
    def init_chat(first_load_state, language):
        welcome_message = "👋 Welcome to Adam Lindberg's resumé chatbot! I can tell you a little about Adam, his skills and projects. What would you like to know?"
        
        if language != "English":
            translate_prompt = f"Translate the following text to {language}. Only provide the translation, nothing else: {welcome_message}"
            for response in stream_claude_3_7_sonnet(translate_prompt):
                welcome_message = response
        
        if first_load_state:
            return [[None, welcome_message]], False
        return [], first_load_state
    
    def bot(history, language):
        user_message = history[-1][0]
        history[-1][1] = ""
        
        # Get the response in English first
        english_response = ""
        for response in stream_claude_3_7_sonnet(user_message):
            english_response = response
        
        # If language is not English, translate the response
        if language != "English":
            translate_prompt = f"Translate the following text to {language}. Only provide the translation, nothing else: {english_response}"
            
            for response in stream_claude_3_7_sonnet(translate_prompt):
                history[-1][1] = response
                yield history
        else:
            # Just return the English response
            history[-1][1] = english_response
            yield history
    
    def translate_response(history, language):
        if not history or not history[-1][1]:
            return history
        
        original_response = history[-1][1]
        translate_prompt = f"Translate the following text to {language}. Only provide the translation, nothing else: {original_response}"
        
        translated_response = ""
        for response in stream_claude_3_7_sonnet(translate_prompt):
            translated_response = response
        
        new_history = history.copy()
        new_history.append([f"Translate to {language}", translated_response])
        
        return new_history
    
    def update_language(language):
        return language
    
    chat.load(init_chat, [first_load, default_language], [chatbot, first_load])
    
    submit_btn.click(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, [chatbot, default_language], chatbot
    )
    
    set_language_btn.click(update_language, language_dropdown, default_language).then(
        init_chat, [first_load, default_language], [chatbot, first_load]
    )
    
    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, [chatbot, default_language], chatbot
    )
    
    
    
    translate_button.click(
        translate_response, 
        [chatbot, language_dropdown], 
        [chatbot]
    )
    
    
    clear.click(lambda: None, None, chatbot, queue=False)

chat.queue()
chat.launch()

/var/folders/w2/nyq_87_d7l5dzyd9hf6z84lw0000gn/T/ipykernel_60301/3548614344.py:5: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7895

To create a public link, set `share=True` in `launch()`.
